In [16]:
# Import the required packages.

import numpy as np
import os

In [17]:
from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

C:\Users\Mussa\AppData\Roaming\Python\Python39\site-packages\tensorflowjs\write_weights.py:28: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  np.uint8, np.uint16, np.bool, np.object]


AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [10]:
from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')

from absl import logging
logging.set_verbosity(logging.ERROR)

In [8]:
"""
   Train the object detection model
   Step 1: Load the dataset
   Images in train_data is used to train the custom object detection model.
   Images in val_data is used to check if the model can generalize well to new images that it hasn't seen before.
"""

train_data = object_detector.DataLoader.from_pascal_voc(
    'datasets/android_figurine/train',
    'datasets/android_figurine/train',
    ['android', 'pig_android']
)

val_data = object_detector.DataLoader.from_pascal_voc(
    'datasets/android_figurine/validate',
    'datasets/android_figurine/validate',
    ['android', 'pig_android']
)

NameError: name 'object_detector' is not defined

In [ ]:
"""
   Step 2: Select a model architecture
   EfficientDet-Lite[0-4] are a family of mobile/IoT-friendly object detection models derived from the EfficientDet architecture.
"""

spec = model_spec.get('efficientdet_lite0')

In [ ]:
"""
   Step 3: Train the TensorFlow model with the training data.
   Set epochs = 20, which means it will go through the training dataset 20 times. You can look at the validation accuracy during training and stop when you see validation loss (val_loss) stop decreasing to avoid overfitting.
   Set batch_size = 4 here so you will see that it takes 15 steps to go through the 62 images in the training dataset.
   Set train_whole_model=True to fine-tune the whole model instead of just training the head layer to improve accuracy. The trade-off is that it may take longer to train the model.
"""

model = object_detector.create(
    train_data, 
    model_spec=spec, 
    batch_size=4, 
    train_whole_model=True, 
    epochs=20, 
    validation_data=val_data
)

In [ ]:
"""
   Step 4. Evaluate the model with the validation data.
   After training the object detection model using the images in the training dataset, use the 10 images in the validation dataset to evaluate how the model performs against new data it has never seen before.

   As the default batch size is 64, it will take 1 step to go through the 10 images in the validation dataset.

   The evaluation metrics are same as COCO.
"""

model.evaluate(val_data)

In [ ]:
"""
   Step 5: Export as a TensorFlow Lite model.
   Export the trained object detection model to the TensorFlow Lite format by specifying which folder you want to export the quantized model to. 
   The default post-training quantization technique is full integer quantization. 
   This allows the TensorFlow Lite model to be smaller, run faster on Raspberry Pi CPU and also compatible with the Google Coral EdgeTPU.
"""

model.export(export_dir='./models/', tflite_filename='android.tflite')

In [ ]:
"""
   Step 6: Evaluate the TensorFlow Lite model.
   Several factors can affect the model accuracy when exporting to TFLite:

   Quantization helps shrinking the model size by 4 times at the expense of some accuracy drop.
   The original TensorFlow model uses per-class non-max supression (NMS) for post-processing, while the TFLite model uses global NMS that's much faster but less accurate. 
   Keras outputs maximum 100 detections while tflite outputs maximum 25 detections.
   Therefore you'll have to evaluate the exported TFLite model and compare its accuracy with the original TensorFlow model.
"""

model.evaluate_tflite('android.tflite', val_data)